# 5.2 Sets of numbers

In [1]:
# install dependencies
%pip install -q amplpy

from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=['base'],  # modules to install
    license_uuid='default',  # license to use
)  # instantiate AMPL object and register magics

Note: you may need to restart the kernel to use updated packages.


Set members may also be numbers. In fact a set's members may be a mixture of numbers
and strings, though this is seldom the case. In an AMPL model, a literal number is
written in the customary way as a sequence of digits, optionally preceded by a sign, containing
an optional decimal point, and optionally followed by an exponent; the exponent
consists of a `d`, `D`, `e`, or `E`, optionally a sign, and a sequence of digits. A number (`1`) and
the corresponding string (`"1"`) are distinct; by contrast, different representations of the
same number, such as `100` and `1E+2`, stand for the same set member.

A set of numbers is often a sequence that corresponds to some progression in the situation
being modeled, such as a series of weeks or years. Just as for strings, the numbers
in a set can be specified as part of the data, or can be specified within a model as a list
between braces, such as `{1,2,3,4,5,6}`. This sort of set can be described more concisely
by the notation `1..6`. An additional `by` clause can be used to specify an interval
other than 1 between the numbers; for instance,
```
2010 .. 2040 by 5
```
represents the set
```
{2010, 2015, 2020, 2025, 2030, 2035, 2040}
```
This kind of expression can be used anywhere that a set is appropriate, and in particular
within the assignment phrase of a set declaration:
```
set YEARS = 2010 .. 2040 by 5;
```
By giving the set a short and meaningful name, this declaration may help to make the rest
of the model more readable.

It is not good practice to specify all the numbers within a `..` expression by literals
like `2040` and `5`, unless the values of these numbers are fundamental to the model or will
rarely change. A better arrangement is seen in the multiperiod production example of
[Figures 4-4](../04/4_3_a_model_of_production_and_transportation.ipynb#fig-4-4) and [4-5](../04/4_3_a_model_of_production_and_transportation.ipynb#fig-4-5), where a parameter `T` is declared to represent the number of periods,
and the expressions `1..T` and `0..T` are used to represent sets of periods over which
parameters, variables, constraints and sums are indexed. The value of `T` is specified in
the data, and is thus easily changed from one run to the next. As a more elaborate example,
we could write
```
param start integer;
param end > start integer;
param interval > 0 integer;
set YEARS = start .. end by interval;
```
If subsequently we were to give the data as
```
ampl.param['start'] = 2010
ampl.param['end'] = 2040
ampl.param['interval'] = 5
```
<a id=p76></a>

then `YEARS` would be the same set as in the previous example (as it would also be if end
were `2043`.) You may use any arithmetic expression to represent any of the values in
a .. expression.

The members of a set of numbers have the same properties as any other numbers, and
hence can be used in arithmetic expressions. A simple example is seen in [Figure 4-4](../04/4_3_a_model_of_production_and_transportation.ipynb#fig-4-4),
where the material balance constraint is declared as
```
subject to Balance {p in PROD, t in 1..T}:
   Make[p,t] + Inv[p,t-1] = Sell[p,t] + Inv[p,t];
```
Because `t` runs over the set `1..T`, we can write `Inv[p,t-1]` to represent the inventory
at the end of the previous week. If `t` instead ran over a set of strings, the expression `t-1`
would be rejected as an error.

Set members need not be integers. AMPL attempts to store each numerical set member
as the nearest representable floating-point number. You can see how this works out
on your computer by trying an experiment like the following:

In [2]:
ampl.option['display_width'] = 50
ampl.display('-5/3 .. 5/3 by 1/3')

set -5/3 .. 5/3 by 1/3 :=
-1.6666666666666667      0.33333333333333326
-1.3333333333333335      0.6666666666666663
-1                       0.9999999999999998
-0.6666666666666667      1.3333333333333333
-0.3333333333333335      1.6666666666666663
-2.220446049250313e-16;



You might expect `0` and `1` to be members of this set, but things do not work out that way
due to rounding error in the floating-point computations. It is unwise to use fractional
numbers in sets, if your model relies on set members having precise values. There should
be no comparable problem with integer members of reasonable size; integers are represented
exactly for magnitudes up to `2^53` (approximately `10^16`) for IEEE standard arithmetic,
and up to `2^47` (approximately `10^14`) for almost any computer in current use.